In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import cv2

In [2]:
def create_dataset():
    # clone repo
    !git clone https://github.com/microsoft/FERPlus.git
    !mkdir ./FER2013plus
    # Extract the downloaded data into another directory
    !tar xvzf "../input/challenges-in-representation-learning-facial-expression-recognition-challenge/fer2013.tar.gz" -C "./FER2013plus/"
    # Run the data generation script on the extracted data
    !python FERPlus/src/generate_training_data.py -d ./FERPlus/data -fer "FER2013plus/fer2013/fer2013.csv" -ferplus "./FERPlus/fer2013new.csv"
create_dataset()

In [3]:
col_names = ["usage","img_name", "neutral","happiness","surprise","sadness","anger","disgust","fear","contempt","Unknown", "NF"]
di = {'neutral': 0, 'happiness': 1,'surprise': 2,'sadness': 3,'anger': 4,'disgust': 5,'fear': 6,'contempt': 7,'Unknown': 8,'NF': 9}
df = pd.read_csv('./FERPlus/fer2013new.csv',names=col_names,header=None,skiprows = 1)
print(df.shape)

In [4]:
def process_df(folder_df):
  use_cols = [x for x in folder_df.columns][2:]
  folder_df['emotion'] = folder_df[use_cols].idxmax(axis=1)
  folder_df = folder_df[['img_name','emotion']]
  folder_df['emotion'] = folder_df['emotion'].map(di)
  folder_df['emotion'] = folder_df['emotion'].apply(str)
  return folder_df

new_col_names = ["img_name","size", "neutral","happiness","surprise","sadness","anger","disgust","fear","contempt","Unknown", "NF"]
train_df =  pd.read_csv('./FERPlus/data/FER2013Train/label.csv',names=new_col_names,header=None)
test_df =  pd.read_csv('./FERPlus/data/FER2013Test/label.csv',names=new_col_names,header=None)
valid_df =  pd.read_csv('./FERPlus/data/FER2013Valid/label.csv',names=new_col_names,header=None)

train_df = process_df(train_df)
test_df = process_df(test_df)
valid_df = process_df(valid_df)

In [5]:
# Preview image
img = cv2.imread('./FERPlus/data/FER2013Train/fer0000000.png')
plt.imshow(img)
print(img.shape)

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1/255,
                              rotation_range=5,
                              shear_range=0.1,
                              zoom_range=0.1,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              horizontal_flip=True,
                              fill_mode='nearest'
                              )


train_generator=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="./FERPlus/data/FER2013Train",
    x_col="img_name",
    y_col="emotion",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(48,48))

valid_datagen = ImageDataGenerator(rescale=1/255)
valid_generator=valid_datagen.flow_from_dataframe(
    dataframe=valid_df,
    directory="./FERPlus/data/FER2013Valid",
    x_col="img_name",
    y_col="emotion",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(48,48))

test_datagen = ImageDataGenerator(rescale=1/255)
test_generator=test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory="./FERPlus/data/FER2013Test",
    x_col="img_name",
    y_col="emotion",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(48,48))


In [7]:
df_copy = df.copy()
useful_cols = [x for x in df_copy.columns][2:]
dfc = df_copy[useful_cols]
dfc['emotion'] = dfc.idxmax(axis=1)
df_final = pd.concat([df_copy[['usage','img_name']],  dfc[['emotion']]], axis=1, ignore_index=False)
df_final['img_name'] = df_final['img_name'].apply(str)
df_final = df_final.replace('nan', np.nan)
df_final.dropna(subset = ["img_name"], inplace=True)
df_final = df_final.reset_index(drop=True)

df_final['emotion'] = df_final['emotion'].map(di)
df_final['emotion'] = df_final['emotion'].apply(str)

df_final_test_generator=test_datagen.flow_from_dataframe(
    dataframe=df_final,
    directory="./FERPlus/data/FER2013Test",
    x_col="img_name",
    y_col="emotion",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(48,48))

In [8]:
train_df.emotion.value_counts().sort_index(ascending=False).plot(kind='bar')

In [9]:
valid_df.emotion.value_counts().sort_index(ascending=False).plot(kind='bar')

In [10]:
test_df.emotion.value_counts().sort_index(ascending=False).plot(kind='bar')

## Transfer learning

In [11]:
#importing mobilenet_v2
mobilnet_model = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape =(48,48,3),
    alpha=1.0, include_top=False, weights='imagenet', pooling=None,     
)

In [12]:
mobilnet_model.summary()

In [13]:
from keras import Sequential
from keras.layers import Dense,Flatten,Dropout,MaxPooling2D,Conv2D,GlobalMaxPooling2D,BatchNormalization
np.random.seed(42)
tf.random.set_seed(42)


model = Sequential([
    mobilnet_model,
    GlobalMaxPooling2D(),
    Dense(1048, activation='relu'),
    Dropout(0.9),
    Dense(1048, activation='relu'),
    Dense(10, activation='softmax')
])
model.summary()

learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
model.compile(optimizer = optimizer, 
              loss = tf.keras.losses.CategoricalCrossentropy(), 
              metrics = ['accuracy'])

STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size

In [14]:
history = model.fit(x = train_generator,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = valid_generator,
          validation_steps = STEP_SIZE_VALID,
          epochs = 50)

In [15]:
#validation and training loss vs epoch graph
history = history.history
n_epochs = len(history['loss'])

plt.figure(figsize=[14,4])
plt.subplot(1,2,1)
plt.plot(range(1, n_epochs+1), history['loss'], label='Training')
plt.plot(range(1, n_epochs+1), history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(range(1, n_epochs+1), history['accuracy'], label='Training')
plt.plot(range(1, n_epochs+1), history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.show()

In [16]:
# save and quantize model
model.save('./emotion.h5')
model = tf.keras.models.load_model('./emotion.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("emotion.tflite", "wb").write(tflite_model)

In [17]:
model.evaluate(df_final_test_generator)